In [ ]:
import pandas as pd
import sys

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# Corrected relative path to the CSV file

In [ ]:
file_path = r'..\data\DataDegreeBeth.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)
display(df_data)

# Instantiate the anonymizer

In [ ]:
anonymizer = DataAnonymizer(df_data, sensitive_columns=['Subgroup1', 'Subgroup2'], frequency='Counts')

# Apply Redact Threshold

In [ ]:
df_redact_threshold = anonymizer.redact_threshold()
display(df_redact_threshold)

In [ ]:
df_redact_threshold.loc[:, 'Overlapping'] = 0

In [ ]:
for organization_column in organization_list:
    for sensitive_column in sensitive_list:
        redacted_filter = df_redact_threshold['RedactBinary'] == 1
        null_filter = (~((df_redact_threshold[sensitive_column].isna()) & (df_redact_threshold[organization_column].isna())))
        filter = (redacted_filter & null_filter)
        df_redacted = df_redact_threshold[filter][[organization_column, sensitive_column]].drop_duplicates()
        df_redacted['OverlappingAdd'] = 1
        df_redact_threshold = df_redact_threshold.merge(df_redacted, on=[organization_column, sensitive_column], how='left')
        df_redact_threshold['OverlappingAdd'].fillna(0, inplace=True)
        df_redact_threshold.loc[~redacted_filter, 'Overlapping'] = df_redact_threshold['Overlapping'] + df_redact_threshold['OverlappingAdd']
        df_redact_threshold.drop('OverlappingAdd', axis=1, inplace=True)
for sensitive_column in sensitive_list:
    redacted_filter = df_redact_threshold['RedactBinary'] == 1
    null_filter = df_redact_threshold[sensitive_column].notna()
    filter = (redacted_filter & null_filter)
    df_redacted = df_redact_threshold[filter][[sensitive_column]].drop_duplicates()
    df_redacted['OverlappingAdd'] = 1
    df_redact_threshold = df_redact_threshold.merge(df_redacted, on=[sensitive_column], how='left')
    df_redact_threshold['OverlappingAdd'].fillna(0, inplace=True)
    df_redact_threshold.loc[~redacted_filter, 'Overlapping'] = df_redact_threshold['Overlapping'] + df_redact_threshold['OverlappingAdd']
    df_redact_threshold.drop('OverlappingAdd', axis=1, inplace=True)
for organization_column in organization_list:
    redacted_filter = df_redact_threshold['RedactBinary'] == 1
    null_filter = df_redact_threshold[organization_column].notna()
    filter = (redacted_filter & null_filter)
    df_redacted = df_redact_threshold[filter][[organization_column]].drop_duplicates()
    df_redacted['OverlappingAdd'] = 1
    df_redact_threshold = df_redact_threshold.merge(df_redacted, on=[organization_column], how='left')
    df_redact_threshold['OverlappingAdd'].fillna(0, inplace=True)
    df_redact_threshold.loc[~redacted_filter, 'Overlapping'] = df_redact_threshold['Overlapping'] + df_redact_threshold['OverlappingAdd']
    df_redact_threshold.drop('OverlappingAdd', axis=1, inplace=True)
display(df_redact_threshold)

In [ ]:
df_min = df_redact_threshold[df_redact_threshold['RedactBinary'] != 1].groupby('RedactBinary').min().reset_index()
minimum_value = df_min['Overlapping'].loc[0]
df_max = df_redact_threshold[df_redact_threshold['RedactBinary'] != 1].groupby('RedactBinary').max().reset_index()
display(df_max)
max_value = df_max['Overlapping'].loc[0]


In [ ]:
print(minimum_value)
print(max_value)

In [ ]:
df_redact_threshold.to_csv('RedactThreshold.csv')